# Calculate Q-Rookley

- Input data "20172022_processed_1_3_4.csv", from 2017/7/1 to 2022/12/31, same as used in Figlewski

- Use 500 rolling window for P density (when calculating EPK)

- Use logreturn instead of log+1

- Use 
```
r.source('Q_Rookley_main_3.R')
```
and
```
'EPK_library_1.r'
```
to calculate Q_Rookley, along with 
```
get_btc_prices_2015_2022
```
for physical density.

- Interest rate: 0

- Moneyness range is [min moneyness, max moneyness]

- Bandwidth 0.3

- tau=18

In [1]:
import platform
plat = platform.system()
import locale
if plat == 'Darwin':
    print('Using Macos Locale')
    locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
else:
    locale.setlocale(locale.LC_ALL, 'en_US.utf8')
import datetime
import pandas as pd
import sys
import matplotlib.pyplot as plt
import csv
import gc
import math
import numpy as np
import os
import pdb
import pickle
import statsmodels.api as sm
import time


import rpy2.robjects.packages as rpackages
from rpy2 import robjects

from scipy import stats
from scipy.integrate import simps
from scipy import interpolate

from src.helpers import save_dict, load_dict
from src.brc import BRC
from src.strategies import IronCondor
from src.quandl_data import get_btc_prices_2015_2022

from numpy.lib.function_base import insert

from statsmodels.nonparametric.kernel_regression import KernelReg as nadarajawatson

from sklearn.neighbors import KernelDensity

from pathlib import Path

def filter_sub(_df,  mindate, maxdate, tau):
    # Subset

    # Only calls, tau in [0, 0.25] and fix one day (bc looking at intra day here)
    # (_df['is_call'] == 1) & --> Dont need to filter for calls because that happens in the R script
    # Also need to consider put-call parity there
    sub = _df[(_df['date'] >= mindate) & (_df['date'] <= maxdate) & (_df['moneyness'] >= 0.5) & (_df['moneyness'] < 2) &(_df['mark_iv'] > 0)]# &

    if tau > 0:
        sub = sub[sub['tau'] == tau]

    nrows = sub.shape[0]
    if nrows == 0:
        raise(ValueError('Sub is empty'))
   
    sub['moneyness'] = round(sub['moneyness'], 3)
    sub['underlying_price'] = round(sub['underlying_price'], 2)

    sub = sub.drop_duplicates()
    
    print(sub.describe())

    if nrows > 50000:
        print('large df, subsetting')
        sub = sub.sample(10000)
        print(sub.describe())

    return sub

def prepare_confidence_band_data(df):

            # Old
                # Save for Confidence Band 
                    # Column 1 - Day
                    # Column 2 - Month
                    # Column 3 - Year
                    # Column 4 - Call Indicator
                    # Column 5 - Time Maturity in Days
                    # Column 6 - Strike
                    # Column 7 - IV
                    # Column 8 - Spot
                    # Column 9 - Risk Free Rate
                    # Column 10 - 
                    # Column 11 - Forward
                # 3	1	1997	0	49	3025	165.8	2861.706	0.031465	0.150026951	2863.26
                # Day Month Year Put Days Strike IV     Spot         Risk Free    Tau in Days (255)       Forward

    """
    # Mapping Table for Input

    # Column ... X
    # 1 - date
    # 2 - IV
    # 3 - 1 for Call, 0 for Put
    # 4 - Tau 
    # 5 - Strike
    # 6 - assigned meanwhile as put2call[,6]+ put2call[,7] -  put2call[,5]*exp(- mean(put2call[,8])* tau);
    # 7 - Spot
    # 8 - Probably Interest Rate
    # 9 - assigned meanwhile Moneyness: Spot/Strike
    """
    b = df[['date', 'mark_iv', 'putcall', 'tau', 'K', 'underlying_price', 'interest_rate', 'moneyness', 'instrument_price']]
    b['placeholder'] = 0
    b = b.reset_index(drop=True)
    # b['interest_rate'] = 0.00
    b['forward'] = 1

    # Reorder
    df_c = b[['date', 'mark_iv', 'putcall', 'tau', 'K', 'instrument_price', 'underlying_price', 'interest_rate', 'moneyness']]

    unique_maturities = df.days_to_maturity.unique()[0]
    fname = str('confidence_band_input_') + str(unique_maturities) + str('.csv')
    df_c.to_csv(fname, index = False)
    return fname

def bootstrap(conf_fname, phys_fname, rdate, tau, simmethod, r, out_dir, bw, lower, upper):
                            
    try:
        # MAKE SURE TO USE DIFFERENT 2ND INPUT FOR SP500!!
        moneyness, spd, epk, epk_lo, epk_up, logret, spd_logret, epk_logret, epk_logret_lo, epk_logret_up, epk_BS, tau, volas, spd_lo, spd_up, cdf_m, cdf_ret= r.bootstrap_epk(robjects.StrVector([conf_fname]), 
                                      robjects.StrVector([phys_fname]),
                                      robjects.StrVector(rdate),
                                      robjects.StrVector([out_dir]),
                                      robjects.FloatVector([bw]),
                                      robjects.FloatVector([lower]),
                                      robjects.FloatVector([upper])
                                     )
        # Can recover PD here as SPD/EPK
        spd_df = pd.DataFrame({'m': moneyness,
                               'spdy': spd,
                               'spdy_up': spd_up,
                               'spdy_dn': spd_lo,
                               'epk': epk,
                               'ret': logret,
                               'spd_ret': spd_logret,
                               'cb_ret_up': epk_logret_up,
                               'cb_ret_dn': epk_logret_lo,
                               'epk_ret': epk_logret,
                               'volatility': volas,
                               'epk_up': epk_up,
                               'epk_dn': epk_lo,
                               'cdf_m': cdf_m,
                               'cdf_ret': cdf_ret})

        return spd_df
    
    except Exception as e:
        print('Sim failed, proceeding with the next one... - ', simmethod)
        print('exception: ', e)
        
# Initiate R Objects
base = rpackages.importr("base") 
r = robjects.r
r.source('src/confidence_bands.R')
r.source('src/Q_Rookley_main_3.R')

# BTC prices
tdat = get_btc_prices_2015_2022()
tdat[::-1].to_csv('Data/BTC_USD_Quandl_2015_2022.csv', index = False)

bw_range = np.array([0.3])
r_bandwidth = bw_range[0]

simmethod = ['oldschool']

df = pd.read_csv('Data/20172022_processed_1_3_4.csv')
interest_rate_data = pd.read_csv('Data/IR_daily.csv')
interest_rate_data = interest_rate_data.rename(columns={'index': 'date','DTB3': 'interest_rate'})

df1=df[df['tau']==18]
dates=df1['date'].unique()
#df1=df[df['date']>=dates[9]]
#dates = df1['date'].unique()

dates1 = dates

error_i = []
for i0 in range(dates1.size):
    df2 = df1[df1['date']==dates1[i0]]
    
    curr_day = datetime.datetime.strptime(dates1[i0],'%Y-%m-%d')
    curr_day_starttime = curr_day.replace(hour = 0, minute = 0, second = 0, microsecond = 0)
    curr_day_endtime = curr_day.replace(hour = 23, minute = 59, second = 59, microsecond = 0)
    print(curr_day)

    df3 = pd.merge(df2, interest_rate_data, on='date', how='left')
    

    df3['interest_rate'] = 0
    df3['days_to_maturity'] = df3['tau']
    df3['tau'] = df3['tau']/365
    df3['underlying_price'] = df3['BTC_price']
    df3['instrument_price'] = df3['option_price']
    df3['putcall'][df3['putcall']=='C']=1
    df3['putcall'][df3['putcall']=='P']=0
    
    df_1 = df3[['date','BTC_price', 'K', 'interest_rate', 'tau', 'putcall', 'IV',  'moneyness',
                'days_to_maturity',  'underlying_price', 'instrument_price']]
    df_1['mark_iv']=df_1['IV']/100
    
    df_1.to_csv('Results/option_' + datetime.datetime.strftime(curr_day_starttime, '%Y%m%d') + '.csv')
    
    conf_fname = prepare_confidence_band_data(df_1)
    
    rdate = base.as_Date(curr_day.strftime('%Y-%m-%d'))
    rdate_f = base.format(rdate, '%Y-%m-%d')
    
    tau = df['tau'].unique()[0]
    
    spd_btc = bootstrap(conf_fname, 'Data/BTC_USD_Quandl_2015_2022.csv', rdate_f, tau, simmethod, r, 
                                     'Results/Rookley_Q_CB_1722_1_2_5_1_ttm18', r_bandwidth, df_1['moneyness'].min(), df_1['moneyness'].max())
    
    try :
        
        spd_btc.to_csv('Results/Rookley_Q_CB_1722_1_2_5_1_ttm18/btc_pk_'  + str(curr_day_starttime.__format__('%Y-%m-%d')) +'_bw_' + str(r_bandwidth) +  '.csv')
        
        print(spd_btc)
        
    except Exception as e:
        error_i.append(i0)

Using Macos Locale


/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
R[write to console]: 
Attaching package: ‘matlab’


R[write to console]: The following object is masked from ‘package:stats’:

    reshape


R[write to console]: The following objects are masked from ‘package:utils’:

    find, fix


R[write to console]: The following object is masked from ‘package:base’:

    sum


R[write to console]: KernSmooth 2.23 loaded
Copyright M. P. Wand 1997-2009

2017-07-10 00:00:00


/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_22069/3868568208.py:197: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='C']=1
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_22069/3868568208.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='P']=0
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_22069/3868568208.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

[1] "bandwidth: 0.3"
[1] "test no arbitrage condition"
[1] 44.694713  4.722982 66.258718
                [,1]      [,2]         [,3]     [,4]
sigmas -0.2916035577 0.9722130 -1.082415527 7.025423
       -0.2866314429 0.9680440 -1.051405745 6.912942
       -0.2816839278 0.9640023 -1.020876005 6.800715
       -0.2767607702 0.9600860 -0.990830054 6.688782
       -0.2718617313 0.9562936 -0.961271361 6.577182
       -0.2669865761 0.9526233 -0.932203119 6.465955
       -0.2621350727 0.9490732 -0.903628242 6.355142
       -0.2573069928 0.9456417 -0.875549360 6.244783
       -0.2525021114 0.9423268 -0.847968825 6.134917
       -0.2477202064 0.9391267 -0.820888701 6.025585
       -0.2429610593 0.9360394 -0.794310765 5.916827
       -0.2382244544 0.9330631 -0.768236505 5.808683
       -0.2335101793 0.9301958 -0.742667119 5.701193
       -0.2288180243 0.9274354 -0.717603512 5.594397
       -0.2241477828 0.9247800 -0.693046293 5.488335
       -0.2194992511 0.9222275 -0.668995773 5.383047
       -0.

...

[1] 44.694713  4.722982 66.258718
               [,1]      [,2]         [,3]      [,4]
sigmas -0.565408915 1.4410522 -2.141428022 1.2218983
       -0.555537110 1.4293190 -2.138552302 1.2919501
       -0.545761805 1.4175865 -2.135193037 1.3614384
       -0.536081133 1.4058579 -2.131352041 1.4303463
       -0.526493278 1.3941358 -2.127031287 1.4986569
       -0.516996478 1.3824234 -2.122232913 1.5663530
       -0.507589020 1.3707234 -2.116959226 1.6334176
       -0.498269237 1.3590388 -2.111212708 1.6998335
       -0.489035512 1.3473726 -2.104996018 1.7655835
       -0.479886268 1.3357276 -2.098311996 1.8306507
       -0.470819975 1.3241067 -2.091163667 1.8950178
       -0.461835142 1.3125130 -2.083554244 1.9586681
       -0.452930317 1.3009492 -2.075487129 2.0215846
       -0.444104090 1.2894183 -2.066965918 2.0837507
       -0.435355083 1.2779232 -2.057994400 2.1451498
       -0.426681958 1.2664667 -2.048576560 2.2057657
       -0.418083410 1.2550517 -2.038716579 2.2655822
       -0.40

        0.354054861 0.7214825  0.654032500 2.5645668
        0.358002763 0.7253425  0.667577690 2.5290516
        0.361935140 0.7292760  0.680885161 2.4932295
        0.365852115 0.7332816  0.693952820 2.4571029
        0.369753806 0.7373576  0.706778592 2.4206742
        0.373640334 0.7415026  0.719360423 2.3839460
        0.377511815 0.7457151  0.731696276 2.3469205
        0.381368365 0.7499933  0.743784139 2.3096004
        0.385210099 0.7543358  0.755622017 2.2719881
        0.389037131 0.7587410  0.767207942 2.2340861
        0.392849573 0.7632071  0.778539970 2.1958970
        0.396647535 0.7677326  0.789616180 2.1574233
        0.400431127 0.7723159  0.800434680 2.1186678
        0.404200457 0.7769553  0.810993607 2.0796331
        0.407955633 0.7816491  0.821291124 2.0403219
        0.411696761 0.7863956  0.831325430 2.0007370
        0.415423944 0.7911931  0.841094752 1.9608812
        0.419137287 0.7960400  0.850597355 1.9207575
        0.422836893 0.8009346  0.859831537 1.8

/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_22069/3868568208.py:197: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='C']=1
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_22069/3868568208.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='P']=0
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_22069/3868568208.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

[1] 44.694713  4.722982 66.258718
               [,1]      [,2]         [,3]     [,4]
sigmas -0.576195708 1.2592787 -2.378699277 4.563153
       -0.568552041 1.2489995 -2.357764379 4.543119
       -0.560966357 1.2388147 -2.336946281 4.523311
       -0.553437782 1.2287237 -2.316243930 4.503732
       -0.545965464 1.2187259 -2.295656269 4.484383
       -0.538548567 1.2088207 -2.275182232 4.465266
       -0.531186276 1.1990076 -2.254820743 4.446384
       -0.523877793 1.1892860 -2.234570718 4.427738
       -0.516622336 1.1796554 -2.214431065 4.409331
       -0.509419141 1.1701151 -2.194400680 4.391164
       -0.502267462 1.1606647 -2.174478448 4.373240
       -0.495166567 1.1513035 -2.154663244 4.355561
       -0.488115738 1.1420311 -2.134953930 4.338129
       -0.481114276 1.1328469 -2.115349355 4.320946
       -0.474161494 1.1237503 -2.095848357 4.304016
       -0.467256720 1.1147409 -2.076449756 4.287340
       -0.460399295 1.1058180 -2.057152362 4.270920
       -0.453588574 1.0969813 

        0.207389764 0.5975909  0.430023713 4.935797
        0.210888376 0.5994917  0.451376760 4.951580
        0.214374791 0.6014846  0.472780721 4.967158
        0.217849093 0.6035697  0.494233184 4.982519
        0.221311366 0.6057470  0.515731650 4.997652
        0.224761692 0.6080166  0.537273540 5.012546
        0.228200155 0.6103786  0.558856192 5.027190
        0.231626836 0.6128330  0.580476861 5.041574
        0.235041815 0.6153798  0.602132719 5.055685
        0.238445171 0.6180189  0.623820855 5.069513
        0.241836983 0.6207504  0.645538272 5.083047
        0.245217330 0.6235741  0.667281892 5.096275
        0.248586289 0.6264900  0.689048550 5.109186
        0.251943936 0.6294980  0.710834998 5.121768
        0.255290347 0.6325978  0.732637903 5.134011
        0.258625597 0.6357895  0.754453846 5.145903
        0.261949759 0.6390728  0.776279321 5.157432
        0.265262909 0.6424474  0.798110737 5.168587
        0.268565117 0.6459132  0.819944416 5.179356
        0.27

In [2]:
print(error_i)

[1, 2, 3, 4, 9, 11, 16, 25, 32, 38, 78, 135]
